<a href="https://colab.research.google.com/github/echung2/echung2/blob/master/2024_DH_%EA%B2%A8%EC%9A%B8%ED%95%99%EA%B5%90_003.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 토픽 모델링 결과 시각화하기
이번 실습에서는 pyLDAVis라는 패키지를 통해 토픽 모델링 결과를 시각화하는 작업을 진행해봅니다.




In [ ]:
!pip install "numpy<1.25" kiwipiepy tomotopy nltk pandas pyldavis
# 재설치 후 Restart Session이 필요할 수 있음

In [ ]:
# 실습에 사용할 패키지들을 로드합니다.
import kiwipiepy
import tomotopy as tp
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyLDAvis # LDA 모델 시각화용 패키지

In [ ]:
# Kiwi는 한국어 형태로 분석에 쓰입니다.
kiwi = kiwipiepy.Kiwi(num_workers=2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# 구글 드라이브에서 naver_shopping.txt를 가져옵니다.
# 혹은 https://drive.google.com/file/d/1UkmkPIMGMj7SUWb_uqhLeKepqwnRls3J/view?usp=drive_link 에서 파일을 다운받은뒤
# 직접 업로드하는 방법도 가능합니다.
FILEID = "1UkmkPIMGMj7SUWb_uqhLeKepqwnRls3J"
FILENAME = "naver_shopping.txt"

!wget --load-cookies ~/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies ~/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id={FILEID}' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id={FILEID}" -O {FILENAME} && rm -rf ~/cookies.txt

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


--2024-01-14 16:25:16--  https://docs.google.com/uc?export=download&confirm=&id=1UkmkPIMGMj7SUWb_uqhLeKepqwnRls3J
Resolving docs.google.com (docs.google.com)... 173.194.211.138, 173.194.211.102, 173.194.211.100, ...
Connecting to docs.google.com (docs.google.com)|173.194.211.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1UkmkPIMGMj7SUWb_uqhLeKepqwnRls3J&export=download [following]
--2024-01-14 16:25:16--  https://drive.usercontent.google.com/download?id=1UkmkPIMGMj7SUWb_uqhLeKepqwnRls3J&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.139.132, 2607:f8b0:400c:c05::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.139.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20623547 (20M) [application/octet-stream]
Saving to: ‘naver_shopping.txt’

naver_shopping.txt  100%[===================>]  19.6

In [ ]:
# naver_shopping.txt 에서 문서들을 읽어옵니다.
docs = [line.split('\t', 1) for line in open('naver_shopping.txt', encoding='utf-8')]
docs = [(int(score), sent) for score, sent in docs]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# 읽어온 문서 10개를 출력해봅니다.
docs[:10]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(5, '배공빠르고 굿\n'),
 (2, '택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고\n'),
 (5, '아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 엉성하긴 하지만 편하고 가성비 최고예요.\n'),
 (2,
  '선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전화했더니 바로주신다했지만 배송도 누락되어있었네요.. 확인안하고 바로 선물했으면 큰일날뻔했네요..이렇게 배송이 오래걸렸으면 사는거 다시 생각했을거같아요 아쉽네요..\n'),
 (5, '민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ\n'),
 (2, '비추합니다 계란 뒤집을 때 완전 불편해요 ㅠㅠ 코팅도 묻어나고 보기엔 예쁘고 실용적으로 보였는데 생각보다 진짜 별로입니다.\n'),
 (1,
  '주문을 11월6에 시켰는데 11월16일에 배송이 왔네요 ㅎㅎㅎ 여기 회사측과는 전화도 안되고 아무런 연락을 받을수가 없으니 답답하신 분들은 다른곳에서 사시는거 추천드립니다\n'),
 (2, '넉넉한 길이로 주문했는데도 안 맞네요 별로예요\n'),
 (2, '보폴이 계속 때처럼 나오다가 지금은 안나네요~\n'),
 (2,
  '110인데 전문속옷브랜드 위생팬티105보다 작은듯해요. 불편해요. 밴딩부분이 다 신축성없는 일반실로 되어있어 빅사이즈임에도 빅사이즈같지않아요. 입고벗을때 편하게 밴딩부분이 늘어나고 입었을때도 밴딩이 잡아주어야하는데 말이죠.\n')]

In [ ]:
# Kiwi 형태소 분석기 기반의 토크나이저입니다.
class KoreanTokenizer:
    def __init__(self, kiwi, allowed_tags:list = None, banned_tags:list = None, stopwords:list = None):
        self.kiwi = kiwi
        assert allowed_tags is None or banned_tags is None, 'allowed_tags and banned_tags cannot be used at the same time.'
        self.allowed_tags = set(allowed_tags) if allowed_tags is not None else None
        self.banned_tags = set(banned_tags) if banned_tags is not None else None
        self.stopwords = set(stopwords or [])

    def __call__(self, texts):
        for tokens in self.kiwi.tokenize(text for text, _ in texts):
            if self.allowed_tags is not None:
                tokens = [(token.tagged_form, token.start, token.len) for token in tokens if token.tag in self.allowed_tags and token.tagged_form not in self.stopwords]
            elif self.banned_tags is not None:
                tokens = [(token.tagged_form, token.start, token.len) for token in tokens if token.tag not in self.banned_tags and token.tagged_form not in self.stopwords]
            else:
                tokens = [(token.tagged_form, token.start, token.len) for token in tokens if token.tagged_form not in self.stopwords]
            yield tokens

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# 형태소 분석기의 품사 태그에 관해서는
# https://github.com/bab2min/kiwipiepy?tab=readme-ov-file#%ED%92%88%EC%82%AC-%ED%83%9C%EA%B7%B8
# 를 참조해주세요.
tokenizer = KoreanTokenizer(
    kiwi,
    banned_tags=['EC', 'EF', 'ETN', 'EP', 'ETM',
                 'SF', 'SP', 'SS', 'SSO', 'SSC', 'SE', 'SO', 'SW', 'SB',
                 'Z_CODA',
                 'JKS', 'JKC', 'JKO', 'JKG', 'JC', 'JKB', 'JKV', 'JKV', 'JKQ', 'JX', 'XSN',
                 'XSV', 'XSA',
                 'VCP', 'VCN', 'VX'],
    stopwords=['하/VV', '되/VV']
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# 토크나이저의 생성 결과 10개를 샘플로 출력합니다.
tokenized_samples = list(tokenizer((sent, None) for score, sent in tqdm(docs[:10])))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 10/10 [00:00<00:00, 15.11it/s]


In [ ]:
tokenized_samples[:10]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[('배/NNP', 0, 1), ('공/NNG', 1, 1), ('빠르/VA', 2, 2), ('굿/NNG', 6, 1)],
 [('택배/NNG', 0, 2),
  ('엉망/NNG', 4, 2),
  ('저희/NP', 10, 2),
  ('집/NNG', 12, 1),
  ('밑/NNG', 14, 1),
  ('층/NNG', 16, 1),
  ('말/NNG', 19, 1),
  ('없이/MAG', 21, 2),
  ('놔두/VV', 24, 2),
  ('가/VV', 27, 1)],
 [('아주/MAG', 0, 2),
  ('좋/VA', 2, 1),
  ('바지/NNG', 6, 2),
  ('정말/MAG', 9, 2),
  ('좋/VA', 12, 1),
  ('2/SN', 15, 1),
  ('개/NNB', 16, 1),
  ('더/MAG', 18, 1),
  ('구매/NNG', 20, 2),
  ('이/MM', 26, 1),
  ('가격/NNG', 27, 2),
  ('대/XPN', 31, 1),
  ('박/NNG', 32, 1),
  ('바느질/NNG', 38, 3),
  ('조금/MAG', 43, 2),
  ('엉성/XR', 46, 2),
  ('편하/VA', 55, 2),
  ('가성비/NNG', 59, 3),
  ('최고/NNG', 63, 2)],
 [('선물/NNG', 0, 2),
  ('빨리/MAG', 6, 2),
  ('받/VV-R', 9, 1),
  ('전달/NNG', 13, 2),
  ('상품/NNG', 22, 2),
  ('머/IC', 29, 1),
  ('그/MM', 30, 1),
  ('컵/NNG', 31, 1),
  ('오/VV', 34, 1),
  ('당황/NNG', 37, 2),
  ('전화/NNG', 45, 2),
  ('바로/MAG', 51, 2),
  ('주/VV', 53, 1),
  ('배송/NNG', 60, 2),
  ('누락/NNG', 64, 2),
  ('확인/NNG', 75, 2),
  ('안/MAG', 77, 1),


In [ ]:
# 토픽 모델링에 사용할 문서들 전체를 Corpus로 묶습니다.
corpus = tp.utils.Corpus(tokenizer=tokenizer)
# 실습 시간 관계상 10000개 문서만 사용합니다.
# 문서가 본문과 평점이 함께 들어가 있으므로 평점은 numeric_metadata로 등록합니다.
corpus.process((content, None, {'numeric_metadata': [score]}) for score, content in tqdm(docs[:10000]))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 10000/10000 [00:14<00:00, 687.08it/s]


10000

In [ ]:
# LDA 토픽 모델을 생성합니다.
# k: 주제의 개수. 연구자가 적절한 수치로 선택해야 합니다. 이에 대해서는 별도로 더 설명할 예정입니다.
# min_df: 이 숫자보다 적은 개수의 문서에만 등장하는 어휘는 학습에서 배제시킵니다.
#         극히 소수의 문서에서만 등장하는 어휘는 최종 결과에 노이즈로만 작용하므로 배제하는게 좋습니다.
# tw: 용어 가중치 부여 방법. tp.TermWeight.ONE, tp.TermWeight.IDF, tp.TermWeight.PMI 중 하나를 선택할 수 있습니다.
#     생략할 경우 tp.TermWeight.ONE이 기본값으로 선택됩니다.
model = tp.LDAModel(k=20, min_df=5, tw=tp.TermWeight.PMI, corpus=corpus)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
model.train(0)
model.summary(topic_word_top_n=10)

<Basic Info>
| LDAModel (current version: 0.12.7)
| 10000 docs, 98015 words
| Total Vocabs: 9560, Used Vocabs: 2331
| Entropy of words: 6.44311
| Entropy of term-weighted words: 7.05097
| Removed Vocabs: <NA>
|
<Training Info>
| Iterations: 0, Burn-in steps: 0
| Optimization Interval: 10
| Log-likelihood per word: -9.53929
|
<Initial Parameters>
| tw: TermWeight.PMI
| min_cf: 0 (minimum collection frequency of words)
| min_df: 5 (minimum document frequency of words)
| rm_top: 0 (the number of top words to be removed)
| k: 20 (the number of topics between 1 ~ 32767)
| alpha: [0.1] (hyperparameter of Dirichlet distribution for document-topic, given as a single `float` in case of symmetric prior and as a list with length `k` of `float` in case of asymmetric prior.)
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| seed: 553662792 (random seed)
| trained in version 0.12.7
|
<Parameters>
| alpha (Dirichlet prior on the per-document topic distributions)
|  [0.1 0.1 0.1 

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
model.train(1000, workers=2, show_progress=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Iteration: 100%|██████████| 1000/1000 [00:21<00:00, 46.40it/s, LLPW: -6.525726]


In [ ]:
model.summary(topic_word_top_n=10)

<Basic Info>
| LDAModel (current version: 0.12.7)
| 10000 docs, 98015 words
| Total Vocabs: 9560, Used Vocabs: 2331
| Entropy of words: 6.44311
| Entropy of term-weighted words: 7.05097
| Removed Vocabs: <NA>
|
<Training Info>
| Iterations: 1000, Burn-in steps: 0
| Optimization Interval: 10
| Log-likelihood per word: -6.52502
|
<Initial Parameters>
| tw: TermWeight.PMI
| min_cf: 0 (minimum collection frequency of words)
| min_df: 5 (minimum document frequency of words)
| rm_top: 0 (the number of top words to be removed)
| k: 20 (the number of topics between 1 ~ 32767)
| alpha: [0.1] (hyperparameter of Dirichlet distribution for document-topic, given as a single `float` in case of symmetric prior and as a list with length `k` of `float` in case of asymmetric prior.)
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| seed: 553662792 (random seed)
| trained in version 0.12.7
|
<Parameters>
| alpha (Dirichlet prior on the per-document topic distributions)
|  [0.0235503

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
topic_term_dists = np.stack([model.get_topic_word_dist(k) for k in range(model.k)])
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in model.docs])
doc_topic_dists /= doc_topic_dists.sum(axis=1, keepdims=True)
doc_lengths = np.array([len(doc.words) for doc in model.docs])
vocab = list(model.used_vocabs)
term_frequency = model.used_vocab_freq

# pyLDAvis를 이용한 시각화 결과를 생성합니다.
# 생성된 파일은 웹 브라우저를 통해 살펴볼 수 있습니다.
prepared_data = pyLDAvis.prepare(
    topic_term_dists,
    doc_topic_dists,
    doc_lengths,
    vocab,
    term_frequency,
    start_index=0,
    sort_topics=False
)
pyLDAvis.save_html(prepared_data, 'ldavis.html')


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
